In [3]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import cv2 as cv
import numpy as np

# Edge detection functions
def canny_edge(img):
    return cv.Canny(img, 100, 200)

def sobel_edge(img):
    grad_x = cv.Sobel(img, cv.CV_16S, 1, 0, ksize=3)
    grad_y = cv.Sobel(img, cv.CV_16S, 0, 1, ksize=3)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    return cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

def laplacian_edge(img):
    return cv.convertScaleAbs(cv.Laplacian(img, cv.CV_16S, ksize=3))

# Define the main application class
class ArtEdgeDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("YSMA Edge Detection")
        self.root.geometry("800x600")

        self.username = tk.StringVar()
        self.password = tk.StringVar()
        self.images = ["images/contemporary/bird_paradise.jpg", 
                       "images/contemporary/durbar.jpg", 
                       "images/contemporary/flexibility.jpg", 
                       "images/contemporary/milkmaids.jpg", 
                       "images/contemporary/widowhood.jpg",
                       "images/modern/dancer.jpg",
                       "images/modern/iya.jpg",
                       "images/modern/landscape_north.jpg",
                       "images/modern/nnamdi.jpg",
                       "images/modern/red_sky.jpg",
                       "images/traditional/benin_head.jpg",
                       "images/traditional/gelede_headdress.jpeg",
                       "images/traditional/igbo_ukwu_art.jpg",
                       "images/traditional/port_soldier.jpg",
                       "images/traditional/yorube_human_face.jpg"]

        self.create_login_screen()

    def create_login_screen(self):
        self.clear_screen()
        tk.Label(self.root, text="Username: admin & Password: admin").pack(pady=5)
        tk.Label(self.root, text="Username").pack(pady=5)
        tk.Entry(self.root, textvariable=self.username).pack(pady=5)
        tk.Label(self.root, text="Password").pack(pady=5)
        tk.Entry(self.root, textvariable=self.password, show="*").pack(pady=5)
        tk.Button(self.root, text="Login", command=self.check_login).pack(pady=20)

    def check_login(self):
        if self.username.get() == "admin" and self.password.get() == "admin":
            self.create_main_screen()
        else:
            messagebox.showerror("Error", "Incorrect Username or Password")

    def create_main_screen(self):
        self.clear_screen()

        canvas = tk.Canvas(self.root)
        scrollbar = tk.Scrollbar(self.root, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)

        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        tk.Label(scrollable_frame, text="Select an Image for Edge Detection").grid(row=0, column=0, columnspan=4, pady=10)

        row, col = 1, 0
        for image_path in self.images:
            self.display_thumbnail(scrollable_frame, image_path, row, col)
            col += 1
            if col == 4:
                col = 0
                row += 1

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

    def display_thumbnail(self, parent, image_path, row, col):
        image = Image.open(image_path)
        image.thumbnail((100, 100))
        tk_image = ImageTk.PhotoImage(image)
        btn = tk.Button(parent, image=tk_image, command=lambda path=image_path: self.open_image_editor(path))
        btn.image = tk_image  # Keep a reference to avoid garbage collection
        btn.grid(row=row, column=col, padx=5, pady=5)

    def open_image_editor(self, image_path):
        self.clear_screen()
        self.image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
        self.image_path = image_path

        self.display_image(self.image)

        tk.Button(self.root, text="Back", command=self.create_main_screen).pack(side="bottom", pady=10)

        tk.Label(self.root, text="Select Edge Detection Operator").pack(pady=10)
        operators = ["Canny", "Sobel", "Laplacian"]
        self.operator = tk.StringVar()
        self.operator.set(operators[0])
        tk.OptionMenu(self.root, self.operator, *operators).pack(pady=10)
        tk.Button(self.root, text="Apply Edge Detection", command=self.apply_edge_detection).pack(pady=10)
        tk.Button(self.root, text="Download Image", command=self.download_image).pack(pady=10)

    def display_image(self, image):
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        tk_image = ImageTk.PhotoImage(image.resize((400, 400), Image.LANCZOS))
        panel = tk.Label(self.root, image=tk_image)
        panel.image = tk_image
        panel.pack()

    def apply_edge_detection(self):
        operator = self.operator.get()
        if operator == "Canny":
            edge_image = canny_edge(self.image)
        elif operator == "Sobel":
            edge_image = sobel_edge(self.image)
        elif operator == "Laplacian":
            edge_image = laplacian_edge(self.image)

        self.show_transformed_image(edge_image)

    def show_transformed_image(self, img):
        new_window = tk.Toplevel(self.root)
        new_window.title("Edge Detection Result")
        new_window.geometry("400x400")

        if isinstance(img, np.ndarray):
            img = Image.fromarray(img)
        tk_image = ImageTk.PhotoImage(img.resize((400, 400), Image.LANCZOS))
        panel = tk.Label(new_window, image=tk_image)
        panel.image = tk_image
        panel.pack()

   

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = ArtEdgeDetectionApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_14268\3533166189.py", line 97, in <lambda>
    btn = tk.Button(parent, image=tk_image, command=lambda path=image_path: self.open_image_editor(path))
  File "C:\Users\user\AppData\Local\Temp\ipykernel_14268\3533166189.py", line 116, in open_image_editor
    tk.Button(self.root, text="Download Image", command=self.download_image).pack(pady=10)
AttributeError: 'ArtEdgeDetectionApp' object has no attribute 'download_image'. Did you mean: 'display_image'?
